In [101]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import seaborn as sns

<br>
There are two files I need to merge:
- the cow file: contains hygiene scores and DHIA data. The name of the file is: df_scoreDHIA_August2020.csv
- the sensor file: contains the selected sensor data. The name of the file is: df_sensor_cow_completedates_August2020.csv
<br><br>
The outcome of this notebook is a file that contains both dataframes merged and it is called: df_sensor_cow_completedates_mergedclinicaldata.csv

### Cow file

In [188]:
file_path = "/Users/gemavidal/Box Sync/Vet/Projects and Papers/2021 Chapter 4 Dissertation High SCC Prediciton Udder Hygiene/df_scoreDHIA_August2020.csv"
df_cow = pd.read_csv(file_path, index_col = False)

In [189]:
df_cow.dtypes

Unnamed: 0                                 int64
DIM at dry-off                           float64
Date                                      object
Day first CM                              object
Dry-off date                              object
Fat                                      float64
Fresh date                                object
Lac                                      float64
Lactation number                         float64
Last CM                                   object
Last move                                 object
Last test-day milk production (lbs)       object
Mun                                      float64
N clinical mastitis current lactation    float64
N days for 1st CM case                   float64
N days from last CM to dry-off           float64
Obs                                       object
Order                                    float64
Pen                                      float64
Pro                                      float64
Sample ID           

In [190]:
#df_cow.loc[(df_cow['dryoffdate_stamp'].isnull())]

In [191]:
df_cow[['cow_id','Dry-off date','date','Date','dryoffdate_stamp','date_stamp']].head(10)

,cow_id,Dry-off date,date,Date,dryoffdate_stamp,date_stamp
0,3565,6/2/20,2-Jun-20,6.2.20,2020-06-02,2020-06-02
1,5435,6/2/20,2-Jun-20,6.2.20,2020-06-02,2020-06-02
2,7373,6/2/20,2-Jun-20,6.2.20,2020-06-02,2020-06-02
3,7282,6/2/20,2-Jun-20,6.2.20,2020-06-02,2020-06-02
4,7107,6/2/20,2-Jun-20,6.2.20,2020-06-02,2020-06-02
5,6941,6/2/20,2-Jun-20,6.2.20,2020-06-02,2020-06-02
6,8872,6/2/20,2-Jun-20,6.2.20,2020-06-02,2020-06-02
7,7069,6/2/20,2-Jun-20,6.2.20,2020-06-02,2020-06-02
8,6849,6/2/20,2-Jun-20,6.2.20,2020-06-02,2020-06-02
9,6989,6/2/20,2-Jun-20,6.2.20,2020-06-02,2020-06-02


In [192]:
df_cow.loc[(df_cow['scc_catg'].isnull())]

,Unnamed: 0,DIM at dry-off,Date,Day first CM,Dry-off date,Fat,Fresh date,Lac,Lactation number,Last CM,...,month,notes,recorder,scc_catg,scc_sample_id,stall_number,year,parity_catg,No_CM_catg,season


I need to create new variables related to the dates, so I will remove some variables first to avoid confusion later

In [193]:
df_cow.drop(['Dry-off date', 'date', 'Date', 'date_stamp'], axis = 1, inplace = True)

In [194]:
df_cow['hour_PCT'] = '13:00:00'
df_cow['date_time_stamp_PCT'] = df_cow['dryoffdate_stamp'] + ' ' + df_cow['hour_PCT']
df_cow.head(5)

,Unnamed: 0,DIM at dry-off,Day first CM,Fat,Fresh date,Lac,Lactation number,Last CM,Last move,Last test-day milk production (lbs),...,recorder,scc_catg,scc_sample_id,stall_number,year,parity_catg,No_CM_catg,season,hour_PCT,date_time_stamp_PCT
0,0,302.0,8/6/19,2.24,8/5/19,4.42,3.0,NaN,4/1/20,47,...,daniela,1,11,51,2020,2,1,spring,13:00:00,2020-06-02 13:00:00
1,1,299.0,NaN,1.36,8/8/19,4.78,2.0,NaN,4/15/20,72,...,daniela,0,29,52,2020,2,0,spring,13:00:00,2020-06-02 13:00:00
2,2,269.0,NaN,0.77,9/7/19,4.86,1.0,NaN,2/26/20,82,...,gema,0,49,53,2020,1,0,spring,13:00:00,2020-06-02 13:00:00
3,3,277.0,12/17/19,1.87,8/30/19,4.98,1.0,12/17/19,3/25/20,72,...,gema,0,33,54,2020,1,1,spring,13:00:00,2020-06-02 13:00:00
4,4,307.0,9/14/19,2.11,7/31/19,4.35,1.0,9/14/19,3/25/20,51,...,daniela,0,12,55,2020,1,1,spring,13:00:00,2020-06-02 13:00:00


In [195]:
#df_cow['dryoffdate_stamp'] = pd.to_datetime(df_cow["dryoffdate_stamp"], format = '%Y-%m-%d')
#df_cow['date'] = pd.to_datetime(df_cow["dryoffdate_stamp"], format = '%Y-%m-%d')

In [196]:
df_cow[['cow_id', 'dryoffdate_stamp', 'date_time_stamp_PCT']].dtypes

cow_id                  int64
dryoffdate_stamp       object
date_time_stamp_PCT    object
dtype: object

In [197]:
#df_cow.head(10)

<br>
## Sensor file

In [198]:
file_path = "/Users/gemavidal/Box Sync/Vet/Projects and Papers/2021 Chapter 4 Dissertation High SCC Prediciton Udder Hygiene/df_sensor_cow_completedates_August2020.csv"
df_sensor = pd.read_csv(file_path, index_col = False)


In [199]:
df_sensor.dtypes

Unnamed: 0                     int64
Active                       float64
Eating                       float64
HighActive                   float64
MeasurementCount             float64
NotActive                    float64
Ruminating                   float64
Temperature                  float64
TimeStamp                     object
Unnamed: 0.1                   int64
Unnamed: 0.1.1               float64
cow_id                         int64
date_time_stamp               object
date_time_stamp_duplicate     object
index                        float64
sensor_day                     int64
sensor_day_PCT                 int64
sensor_hour                    int64
sensor_hour_PCT                int64
dtype: object

In [200]:
df_sensor.loc[(df_sensor['date_time_stamp'].isnull())]

,Unnamed: 0,Active,Eating,HighActive,MeasurementCount,NotActive,Ruminating,Temperature,TimeStamp,Unnamed: 0.1,Unnamed: 0.1.1,cow_id,date_time_stamp,date_time_stamp_duplicate,index,sensor_day,sensor_day_PCT,sensor_hour,sensor_hour_PCT


In [201]:
df_sensor.head(3)

,Unnamed: 0,Active,Eating,HighActive,MeasurementCount,NotActive,Ruminating,Temperature,TimeStamp,Unnamed: 0.1,Unnamed: 0.1.1,cow_id,date_time_stamp,date_time_stamp_duplicate,index,sensor_day,sensor_day_PCT,sensor_hour,sensor_hour_PCT
0,0,1.0,31.0,14.0,60.0,13.0,1.0,24.34,20200514 21:00:00,0,0.0,172,2020-05-14 21:00:00,2020-05-14 14:00:00-07:00,6295.0,14,14,21,14
1,1,3.0,0.0,1.0,60.0,23.0,33.0,27.86,20200514 22:00:00,1,1.0,172,2020-05-14 22:00:00,2020-05-14 15:00:00-07:00,6296.0,14,14,22,15
2,2,9.0,0.0,0.0,60.0,17.0,34.0,29.86,20200514 23:00:00,2,2.0,172,2020-05-14 23:00:00,2020-05-14 16:00:00-07:00,6297.0,14,14,23,16


In [202]:
#df_sensor['date_time_stamp'] = pd.to_datetime(df_sensor['date_time_stamp'], format = '%Y-%m-%d %H:%M:%S')

In [203]:
#df_sensor.head(3)

<br>
## Prepare to merge

### Use Pacific Coast Time zone in both dataframes
<br>

First, make the date_time variables as timestamps

In [204]:
print df_cow['date_time_stamp_PCT'].head(3)
print df_sensor['date_time_stamp'].head(3)

0    2020-06-02 13:00:00
1    2020-06-02 13:00:00
2    2020-06-02 13:00:00
Name: date_time_stamp_PCT, dtype: object
0    2020-05-14 21:00:00
1    2020-05-14 22:00:00
2    2020-05-14 23:00:00
Name: date_time_stamp, dtype: object


In [205]:
df_cow['date_time_stamp_PCT_duplicate'] = pd.to_datetime(df_cow['date_time_stamp_PCT'], format = '%Y-%m-%d %H:%M:%S')
df_sensor['date_time_stamp_duplicate'] = pd.to_datetime(df_sensor['date_time_stamp'], format = '%Y-%m-%d %H:%M:%S')

In [206]:
#df_cow['date_time_stamp_duplicate'] = df_cow['dryoffdate_stamp']
#df_sensor['date_time_stamp_duplicate'] = df_sensor['date_time_stamp']

In [207]:
#df_cow['date_time_stamp_duplicate'].head(3)

<br>Move them to the index because that is the only way to make them timezone aware

In [208]:
df_cow.set_index('date_time_stamp_PCT_duplicate', inplace = True)
df_sensor.set_index('date_time_stamp_duplicate', inplace = True)

<br>
Second, I need to assign a time zone. For df_cow, the timezone is PCT, I just neet to make it time zone aware. For df_sensor, I need to make it UTC time zone aware, then change it to PCT.

In [209]:
df_cow.index = df_cow.index.tz_localize('America/Los_Angeles')
df_cow.reset_index(inplace = True)


df_sensor.index = df_sensor.index.tz_localize('UTC')
df_sensor.index = df_sensor.index.tz_convert('America/Los_Angeles')
df_sensor.reset_index(inplace = True)

<br>
Third, to avoid confusion, rename the column names from 'date_time_stamp_duplicate' to 'date_time_stamp_PCT'

In [210]:
df_cow = df_cow.rename(columns = {'date_time_stamp_PCT_duplicate':'datetime_stamp'})
df_sensor = df_sensor.rename(columns = {'date_time_stamp_duplicate':'datetime_stamp'})

In [211]:
print df_cow['datetime_stamp'].head(3)
print df_sensor['datetime_stamp'].head(3)

0   2020-06-02 13:00:00-07:00
1   2020-06-02 13:00:00-07:00
2   2020-06-02 13:00:00-07:00
Name: datetime_stamp, dtype: datetime64[ns, America/Los_Angeles]
0   2020-05-14 14:00:00-07:00
1   2020-05-14 15:00:00-07:00
2   2020-05-14 16:00:00-07:00
Name: datetime_stamp, dtype: datetime64[ns, America/Los_Angeles]


<br>
Last, extract the date from the new time stamps. The date_PCT will be the index I will use to merge the dataframes

In [212]:
df_cow['date_cowdata_PCT'] = df_cow['datetime_stamp'].dt.date
df_sensor['date_sensordata_PCT'] = df_sensor['datetime_stamp'].dt.date

### Prepare indexes before merging

In [213]:
#df_cow_ts = df_cow.set_index(['cow_id', 'date_PCT'])
df_cow_ts = df_cow.set_index(['cow_id', 'datetime_stamp'])
df_cow_ts.head(3)

,,Unnamed: 0,DIM at dry-off,Day first CM,Fat,Fresh date,Lac,Lactation number,Last CM,Last move,Last test-day milk production (lbs),...,scc_catg,scc_sample_id,stall_number,year,parity_catg,No_CM_catg,season,hour_PCT,date_time_stamp_PCT,date_cowdata_PCT
cow_id,datetime_stamp,,,,,,,,,,,,,,,,,,,,,
3565,2020-06-02 13:00:00-07:00,0,302.0,8/6/19,2.24,8/5/19,4.42,3.0,NaN,4/1/20,47,...,1,11,51,2020,2,1,spring,13:00:00,2020-06-02 13:00:00,2020-06-02
5435,2020-06-02 13:00:00-07:00,1,299.0,NaN,1.36,8/8/19,4.78,2.0,NaN,4/15/20,72,...,0,29,52,2020,2,0,spring,13:00:00,2020-06-02 13:00:00,2020-06-02
7373,2020-06-02 13:00:00-07:00,2,269.0,NaN,0.77,9/7/19,4.86,1.0,NaN,2/26/20,82,...,0,49,53,2020,1,0,spring,13:00:00,2020-06-02 13:00:00,2020-06-02


In [214]:
df_sensor_ts = df_sensor.set_index(['cow_id', 'datetime_stamp'])
df_sensor_ts.head(3)

Unnamed: 0  Active  Eating  HighActive  \
cow_id datetime_stamp                                                      
172    2020-05-14 14:00:00-07:00           0     1.0    31.0        14.0   
       2020-05-14 15:00:00-07:00           1     3.0     0.0         1.0   
       2020-05-14 16:00:00-07:00           2     9.0     0.0         0.0   

                                  MeasurementCount  NotActive  Ruminating  \
cow_id datetime_stamp                                                       
172    2020-05-14 14:00:00-07:00              60.0       13.0         1.0   
       2020-05-14 15:00:00-07:00              60.0       23.0        33.0   
       2020-05-14 16:00:00-07:00              60.0       17.0        34.0   

                                  Temperature          TimeStamp  \
cow_id datetime_stamp                                              
172    2020-05-14 14:00:00-07:00        24.34  20200514 21:00:00   
       2020-05-14 15:00:00-07:00        27.86  20200514 22:00:00   
       2020-05-14 16:00:00-07:00        29.86  20200514 23:00:00   

                                  Unnamed: 0.1  Unnamed: 0.1.1  \
cow_id datetime_stamp                                            
172    2020-05-14 14:00:00-07:00             0             0.0   
       2020-05-14 15:00:00-07:00             1             1.0   
       2020-05-14 16:00:00-07:00             2             2.0   

                                      date_time_stamp   index  sensor_day  \
cow_id datetime_stamp                                                       
172    2020-05-14 14:00:00-07:00  2020-05-14 21:00:00  6295.0          14   
       2020-05-14 15:00:00-07:00  2020-05-14 22:00:00  6296.0          14   
       2020-05-14 16:00:00-07:00  2020-05-14 23:00:00  6297.0          14   

                                  sensor_day_PCT  sensor_hour  \
cow_id datetime_stamp                                           
172    2020-05-14 14:00:00-07:00              14           21   
       2020-05-14 15:00:00-07:00              14           22   
       2020-05-14 16:00:00-07:00              14           23   

                                  sensor_hour_PCT date_sensordata_PCT  
cow_id datetime_stamp                                                  
172    2020-05-14 14:00:00-07:00               14          2020-05-14  
       2020-05-14 15:00:00-07:00               15          2020-05-14  
       2020-05-14 16:00:00-07:00               16          2020-05-14

In [215]:
print df_sensor.shape, df_cow.shape

(47366, 20) (123, 50)


In [216]:
df_sensor_cow = pd.merge(df_sensor_ts, df_cow_ts, left_index=True, right_index=True, how='outer', indicator = True)

In [217]:
print df_sensor_cow.shape

(47386, 67)


In [219]:
#df_sensor_cow.dtypes
df_sensor_cow.reset_index(inplace = True)
df_sensor_cow.head(4)

,cow_id,datetime_stamp,Unnamed: 0_x,Active,Eating,HighActive,MeasurementCount,NotActive,Ruminating,Temperature,...,scc_sample_id,stall_number,year,parity_catg,No_CM_catg,season,hour_PCT,date_time_stamp_PCT,date_cowdata_PCT,_merge
0,172,2020-05-14 14:00:00-07:00,0.0,1.0,31.0,14.0,60.0,13.0,1.0,24.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,172,2020-05-14 15:00:00-07:00,1.0,3.0,0.0,1.0,60.0,23.0,33.0,27.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,172,2020-05-14 16:00:00-07:00,2.0,9.0,0.0,0.0,60.0,17.0,34.0,29.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,172,2020-05-14 17:00:00-07:00,3.0,4.0,19.0,7.0,59.0,0.0,29.0,30.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [226]:
df_sensor_cow.loc[(df_sensor_cow['cow_id']== 172)][['cow_id','datetime_stamp',
                                                    'season','Active',
                                                    'date_time_stamp_PCT','date_time_stamp']]

,cow_id,datetime_stamp,season,Active,date_time_stamp_PCT,date_time_stamp
0,172,2020-05-14 14:00:00-07:00,NaN,1.0,NaN,2020-05-14 21:00:00
1,172,2020-05-14 15:00:00-07:00,NaN,3.0,NaN,2020-05-14 22:00:00
2,172,2020-05-14 16:00:00-07:00,NaN,9.0,NaN,2020-05-14 23:00:00
3,172,2020-05-14 17:00:00-07:00,NaN,4.0,NaN,2020-05-15 00:00:00
4,172,2020-05-14 18:00:00-07:00,NaN,12.0,NaN,2020-05-15 01:00:00
5,172,2020-05-14 19:00:00-07:00,NaN,12.0,NaN,2020-05-15 02:00:00
6,172,2020-05-14 20:00:00-07:00,NaN,4.0,NaN,2020-05-15 03:00:00
7,172,2020-05-14 21:00:00-07:00,NaN,5.0,NaN,2020-05-15 04:00:00
8,172,2020-05-14 22:00:00-07:00,NaN,3.0,NaN,2020-05-15 05:00:00
9,172,2020-05-14 23:00:00-07:00,NaN,11.0,NaN,2020-05-15 06:00:00


In [227]:
# Save the file
df_sensor_cow.to_csv("/Users/gemavidal/Box Sync/Vet/Projects and Papers/2021 Chapter 4 Dissertation High SCC Prediciton Udder Hygiene/df_sensor_cow_completedates_mergedclinicaldata_August2020.csv")


<br>
What I have at this point is a dataframe with one row per hour and day and cow, that includes all the clinical cow data assigned on the dry-off day at 13:00:00 PCT, and also keeps record of the original sensor data on UTC, and the original dry-off date and time in PCT.
<br><br>
It also has the sensor data, which goes beyond the dry-off date/time.